In [4]:
import pandas as pd
import numpy as np

In [5]:
data=pd.read_excel("ENB2012_data.xlsx")
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    int64  
 6   X7      768 non-null    float64
 7   X8      768 non-null    int64  
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


In [7]:
data.shape

(768, 10)

In [8]:
data.isnull().values.any()

False

In [9]:
data.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307195,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090204,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


In [10]:
X = data.drop(['Y1', 'Y2'], axis=1).values
y_1= ((data['Y1']).values).reshape(-1, 1)
y_2= ((data['Y2']).values).reshape(-1, 1)

In [11]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_train , X_test , y_train , y_test = train_test_split(X , y_1 , test_size=0.2 , random_state =42)


In [12]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [13]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
cv=RepeatedKFold(n_splits=10,n_repeats=10,random_state=True)

In [14]:
from numpy import mean
from numpy import std
params=[0.001,0.01,0.1, 1.0, 10.0]
for alpha in params:
    model=Ridge(alpha=alpha)
    model.fit(X_train,y_train)
    print(model.score(X_train,y_train))

model=Ridge(alpha=0.001)
model.fit(X_train,y_train)

0.9170648844382435
0.9170648623919732
0.9170627789106799
0.9169321430078381
0.9143854286804884


Ridge(alpha=0.001)

In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred=model.predict(X_test)
mse=mean_squared_error(y_test,y_pred)
mse

9.153263783926711

In [16]:
y_pred=model.predict(X_test)
mae=mean_absolute_error(y_test,y_pred)
mae

2.1820817190071176

In [17]:
scores = cross_val_score(model, X_train, y_train,scoring="neg_mean_squared_error", cv=cv)

In [18]:
-scores.mean()

8.58963018650108

In [19]:
scores.std()

1.792015520510096

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
parameters={
    'n_estimators':[10,50,100,250,500],
    'max_depth':[50,150,250],
    'min_samples_split':[2,3],
    'min_samples_leaf':[1,2,3]
    }
random_forest=RandomForestRegressor()
grid_search=GridSearchCV(random_forest,param_grid=parameters,cv=cv,n_jobs = -1, verbose = 2)
grid_search.fit(X_train,y_train)
grid_search.best_params_

Fitting 100 folds for each of 90 candidates, totalling 9000 fits


/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/s

[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s


/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/s

[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s

[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s


...

[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   1.5s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   1.3s
[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   1.5s


/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END max_depth=50, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   1.7s


/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/samet/anaconda3/envs/sklearn-env/lib/python3.10/s

KeyboardInterrupt: 